# Thumalien - Notebook d'Exploration

**Projet M1 Data & IA** - Détection de Fake News sur Bluesky

Ce notebook couvre :
1. Connexion et collecte de données via l'API Bluesky
2. Exploration et statistiques descriptives
3. Prétraitement NLP
4. Test du classifieur de fake news
5. Test de l'analyseur d'émotions
6. Explicabilité
7. Suivi énergétique

In [ ]:
import sys
sys.path.insert(0, '..')

import os
import json
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from dotenv import load_dotenv
from collections import Counter

sns.set_theme(style='whitegrid', palette='Set2')
pd.set_option('display.max_colwidth', 120)

load_dotenv('../.env')
print('Setup OK')

---
## 1. Connexion à Bluesky et collecte de données

In [ ]:
from src.collector.bluesky_client import BlueskyCollector

handle = os.getenv('BLUESKY_HANDLE')
password = os.getenv('BLUESKY_PASSWORD')

collector = BlueskyCollector(handle, password)
print(f'Connecté en tant que {handle}')

In [ ]:
# Collecte de posts sur plusieurs thématiques
queries = ['fake news', 'désinformation', 'santé', 'politique', 'climat']
all_posts = []

for query in queries:
    posts = collector.search_posts(query, lang='fr', limit=30)
    for p in posts:
        p['query'] = query
    all_posts.extend(posts)
    print(f'  "{query}" : {len(posts)} posts')

print(f'\nTotal collecté : {len(all_posts)} posts')

In [ ]:
# Conversion en DataFrame
df = pd.DataFrame(all_posts)
print(f'Shape : {df.shape}')
print(f'Colonnes : {list(df.columns)}')
df.head()

---
## 2. Exploration et statistiques descriptives

In [ ]:
# Statistiques de base
print('=== Statistiques des posts ===')
print(f'Posts uniques (par URI) : {df["uri"].nunique()}')
print(f'Auteurs uniques : {df["author_handle"].nunique()}')
print(f'\n--- Engagement ---')
for col in ['like_count', 'repost_count', 'reply_count']:
    print(f'{col}: mean={df[col].mean():.1f}, median={df[col].median():.0f}, max={df[col].max()}')

In [ ]:
# Longueur des textes
df['text_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['text_length'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Nombre de caractères')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution de la longueur des posts')
axes[0].axvline(df['text_length'].mean(), color='red', linestyle='--', label=f'Moyenne: {df["text_length"].mean():.0f}')
axes[0].legend()

axes[1].hist(df['word_count'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Nombre de mots')
axes[1].set_ylabel('Fréquence')
axes[1].set_title('Distribution du nombre de mots')
axes[1].axvline(df['word_count'].mean(), color='red', linestyle='--', label=f'Moyenne: {df["word_count"].mean():.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Répartition par requête de recherche
fig = px.histogram(df, x='query', color='query',
                   title='Nombre de posts collectés par thématique',
                   labels={'query': 'Thématique', 'count': 'Nombre'})
fig.show()

In [ ]:
# Top auteurs
top_authors = df['author_handle'].value_counts().head(15)

fig = px.bar(x=top_authors.values, y=top_authors.index, orientation='h',
             title='Top 15 auteurs les plus actifs',
             labels={'x': 'Nombre de posts', 'y': 'Auteur'})
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

In [ ]:
# Engagement par thématique
engagement = df.groupby('query')[['like_count', 'repost_count', 'reply_count']].mean()

fig = px.bar(engagement, barmode='group',
             title='Engagement moyen par thématique',
             labels={'value': 'Moyenne', 'variable': 'Type'})
fig.show()

---
## 3. Prétraitement NLP

In [ ]:
from src.preprocessing.text_processor import clean_text, tokenize, preprocess_batch

# Exemple sur un post
sample = df.iloc[0]
print('=== Texte original ===')
print(sample['text'])
print('\n=== Texte nettoyé ===')
cleaned = clean_text(sample['text'])
print(cleaned)
print('\n=== Tokens (lemmes) ===')
tokens = tokenize(cleaned, lang='fr')
print(tokens)

In [ ]:
# Prétraitement du dataset complet
posts_list = df.to_dict('records')
processed = preprocess_batch(posts_list)

df_processed = pd.DataFrame(processed)
print(f'Colonnes ajoutées : {[c for c in df_processed.columns if c not in df.columns]}')
df_processed[['text', 'clean_text', 'tokens']].head()

In [ ]:
# Mots les plus fréquents
all_tokens = [token for tokens in df_processed['tokens'] for token in tokens]
token_freq = Counter(all_tokens).most_common(30)

words, counts = zip(*token_freq)
fig = px.bar(x=list(counts), y=list(words), orientation='h',
             title='Top 30 des mots les plus fréquents (après lemmatisation)',
             labels={'x': 'Fréquence', 'y': 'Mot'})
fig.update_layout(yaxis={'categoryorder': 'total ascending'}, height=600)
fig.show()

In [ ]:
# Wordcloud
try:
    from wordcloud import WordCloud

    text_all = ' '.join(all_tokens)
    wc = WordCloud(width=800, height=400, background_color='white',
                   colormap='viridis', max_words=100).generate(text_all)

    plt.figure(figsize=(14, 7))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title('Wordcloud des posts collectés')
    plt.show()
except ImportError:
    print('pip install wordcloud pour afficher le nuage de mots')

---
## 4. Classification Fake News

In [ ]:
from src.models.fake_news_detector import FakeNewsDetector

detector = FakeNewsDetector()
print('Modèle chargé')

In [ ]:
# Test sur quelques exemples
test_texts = [
    "Le président a annoncé de nouvelles mesures économiques lors de la conférence de presse.",
    "BREAKING: Les extraterrestres ont atterri à Paris, le gouvernement cache la vérité !!!",
    "Selon l'OMS, le vaccin est sûr et efficace après les essais cliniques de phase 3.",
    "On vous cache tout ! Les élites contrôlent le monde avec la 5G, réveillez-vous !",
    "Le match de football s'est terminé par un score de 2-1.",
]

print('=== Test du classifieur ===')
for text in test_texts:
    result = detector.predict(text)
    print(f'\n[{result["label"].upper()}] (confiance: {result["confidence"]:.2%})')
    print(f'  Texte: {text[:80]}...')
    print(f'  Scores: {result["scores"]}')

In [ ]:
# Classification de tous les posts collectés
texts = df_processed['clean_text'].tolist()
predictions = detector.predict_batch(texts)

df_processed['cred_label'] = [p['label'] for p in predictions]
df_processed['cred_confidence'] = [p['confidence'] for p in predictions]
df_processed['cred_scores'] = predictions

print('=== Répartition des labels ===')
print(df_processed['cred_label'].value_counts())

In [ ]:
# Visualisation de la répartition
colors = {'fiable': '#2ecc71', 'douteux': '#f39c12', 'fake': '#e74c3c'}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
label_counts = df_processed['cred_label'].value_counts()
axes[0].pie(label_counts.values, labels=label_counts.index,
            colors=[colors[l] for l in label_counts.index],
            autopct='%1.1f%%', startangle=90)
axes[0].set_title('Répartition de la crédibilité')

# Histogramme de confiance
for label in ['fiable', 'douteux', 'fake']:
    subset = df_processed[df_processed['cred_label'] == label]['cred_confidence']
    axes[1].hist(subset, bins=15, alpha=0.6, label=label, color=colors[label])
axes[1].set_xlabel('Score de confiance')
axes[1].set_ylabel('Fréquence')
axes[1].set_title('Distribution des scores de confiance')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Crédibilité par thématique
cross = pd.crosstab(df_processed['query'], df_processed['cred_label'], normalize='index') * 100

fig = px.bar(cross, barmode='stack',
             title='Crédibilité par thématique (%)',
             labels={'value': 'Pourcentage', 'query': 'Thématique'},
             color_discrete_map=colors)
fig.show()

---
## 5. Analyse émotionnelle

In [ ]:
from src.models.emotion_analyzer import EmotionAnalyzer

emotion_analyzer = EmotionAnalyzer()
print('Analyseur d\'émotions chargé')

In [ ]:
# Analyse émotionnelle de tous les posts
emotions = emotion_analyzer.analyze_batch(texts)

df_processed['emotion_dominant'] = [e['dominant_emotion'] for e in emotions]
df_processed['emotion_confidence'] = [e['confidence'] for e in emotions]
df_processed['emotion_scores'] = emotions

print('=== Répartition des émotions ===')
print(df_processed['emotion_dominant'].value_counts())

In [ ]:
# Visualisation des émotions
emo_colors = {
    'colère': '#e74c3c', 'dégoût': '#8e44ad', 'peur': '#2c3e50',
    'joie': '#f1c40f', 'tristesse': '#3498db', 'surprise': '#e67e22', 'neutre': '#95a5a6'
}

fig = px.histogram(df_processed, x='emotion_dominant', color='emotion_dominant',
                   color_discrete_map=emo_colors,
                   title='Distribution des émotions dominantes')
fig.show()

In [ ]:
# Radar chart : profil émotionnel moyen
all_emo_scores = {}
for e in emotions:
    for emo, score in e['scores'].items():
        all_emo_scores.setdefault(emo, []).append(score)

avg_emo = {emo: np.mean(vals) for emo, vals in all_emo_scores.items()}

fig = go.Figure(data=go.Scatterpolar(
    r=list(avg_emo.values()),
    theta=list(avg_emo.keys()),
    fill='toself'
))
fig.update_layout(title='Profil émotionnel moyen des posts collectés',
                  polar=dict(radialaxis=dict(visible=True, range=[0, 1])))
fig.show()

In [ ]:
# Croisement émotions x crédibilité
fig = px.histogram(df_processed, x='emotion_dominant', color='cred_label',
                   color_discrete_map=colors, barmode='group',
                   title='Émotions par catégorie de crédibilité')
fig.show()

In [ ]:
# Heatmap émotions x crédibilité
cross_emo = pd.crosstab(df_processed['cred_label'], df_processed['emotion_dominant'])

plt.figure(figsize=(10, 4))
sns.heatmap(cross_emo, annot=True, fmt='d', cmap='YlOrRd')
plt.title('Heatmap : Crédibilité vs Émotion dominante')
plt.xlabel('Émotion')
plt.ylabel('Crédibilité')
plt.tight_layout()
plt.show()

---
## 6. Explicabilité

In [ ]:
from src.explainability.explainer import PredictionExplainer

explainer = PredictionExplainer(detector.model, detector.tokenizer)

# Analyser les posts classés "douteux" ou "fake"
suspicious = df_processed[df_processed['cred_label'].isin(['douteux', 'fake'])].head(5)

print(f'{len(suspicious)} posts suspects à expliquer\n')

for idx, row in suspicious.iterrows():
    explanation = explainer.explain(row['clean_text'])
    print(f'--- Post #{idx} [{row["cred_label"].upper()}] ---')
    print(f'Texte : {row["text"][:100]}...')
    print(f'Émotion : {row["emotion_dominant"]}')
    print(f'Top mots influents :')
    for w in explanation['top_influential_words'][:5]:
        bar = '█' * int(w['importance_normalized'] * 20)
        print(f'  {w["token"]:>15s} {bar} ({w["importance_normalized"]:.2f})')
    print()

In [ ]:
# Visualisation de l'importance des mots pour un post
if len(suspicious) > 0:
    sample_text = suspicious.iloc[0]['clean_text']
    expl = explainer.explain(sample_text)

    top_words = expl['all_word_importances'][:15]
    tokens_list = [w['token'] for w in top_words]
    importances = [w['importance_normalized'] for w in top_words]

    fig = px.bar(x=importances, y=tokens_list, orientation='h',
                 title=f'Importance des mots (post classé "{suspicious.iloc[0]["cred_label"]}")',
                 labels={'x': 'Importance normalisée', 'y': 'Token'},
                 color=importances, color_continuous_scale='Reds')
    fig.update_layout(yaxis={'categoryorder': 'total ascending'}, height=500)
    fig.show()
else:
    print('Aucun post suspect trouvé pour démontrer l\'explicabilité')

---
## 7. Suivi énergétique (Green IT)

In [ ]:
from src.monitoring.energy_tracker import EnergyTracker
import time

tracker = EnergyTracker(output_dir='../data/monitoring')

# Mesurer l'énergie d'une classification batch
with tracker.track('classification_batch'):
    _ = detector.predict_batch(texts[:20])

# Mesurer l'énergie d'une analyse émotionnelle
with tracker.track('emotion_batch'):
    _ = emotion_analyzer.analyze_batch(texts[:20])

summary = tracker.get_summary()
print('=== Rapport énergétique ===')
print(f'Émissions totales  : {summary["total_emissions_kg_co2"]:.6f} kg CO2')
print(f'Énergie totale     : {summary["total_energy_kwh"]:.6f} kWh')
print(f'Durée totale       : {summary["total_duration_seconds"]:.2f} s')
print(f'Nombre de tâches   : {summary["num_tasks"]}')

In [ ]:
# Visualisation
if summary['tasks']:
    task_df = pd.DataFrame(summary['tasks'])

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    axes[0].bar(task_df['task'], task_df['duration_seconds'], color=['#3498db', '#e74c3c'])
    axes[0].set_ylabel('Durée (secondes)')
    axes[0].set_title('Durée par tâche')

    axes[1].bar(task_df['task'], task_df['emissions_kg_co2'], color=['#2ecc71', '#f39c12'])
    axes[1].set_ylabel('Émissions (kg CO2)')
    axes[1].set_title('Émissions CO2 par tâche')

    plt.tight_layout()
    plt.show()

---
## 8. Résumé et export

In [ ]:
# Résumé global
print('=' * 60)
print('RÉSUMÉ DE L\'EXPLORATION')
print('=' * 60)
print(f'Posts collectés     : {len(df_processed)}')
print(f'Auteurs uniques     : {df_processed["author_handle"].nunique()}')
print(f'Thématiques         : {", ".join(queries)}')
print(f'\n--- Crédibilité ---')
for label in ['fiable', 'douteux', 'fake']:
    count = (df_processed['cred_label'] == label).sum()
    pct = count / len(df_processed) * 100
    print(f'  {label:>10s} : {count:3d} ({pct:.1f}%)')
print(f'\n--- Émotion dominante la plus fréquente ---')
print(f'  {df_processed["emotion_dominant"].mode().iloc[0]}')
print(f'\n--- Énergie ---')
print(f'  CO2 : {summary["total_emissions_kg_co2"]:.6f} kg')
print(f'  kWh : {summary["total_energy_kwh"]:.6f}')

In [ ]:
# Export des résultats d'exploration
export_cols = ['author_handle', 'text', 'clean_text', 'query',
               'like_count', 'repost_count', 'reply_count',
               'cred_label', 'cred_confidence', 'emotion_dominant', 'emotion_confidence']

df_export = df_processed[export_cols]
df_export.to_csv('../data/processed/exploration_results.csv', index=False, encoding='utf-8')
print(f'Résultats exportés : ../data/processed/exploration_results.csv ({len(df_export)} lignes)')